In [1]:
import csv
import pandas as pd
import spacy
import re
import string
import pyLDAvis
import pyLDAvis.sklearn
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
from sklearn.decomposition import LatentDirichletAllocation, NMF

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

nlp = spacy.load("en_core_web_sm")

In [2]:
df = pd.read_csv('genshintwitternouveau.csv')
df = df.drop(columns = ['Unnamed: 0', 'outlinks', 'tcooutlinks'])

alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

def cleanurlhashtags(input):

    text = re.sub(r"http\S+", "", input)
    text = re.sub(r"#\S+", "", text)
    return text

def tostring(input):
    return str(input)

df['content'] = df['content'].apply(tostring)
df['content'] = df['content'].apply(cleanurlhashtags)
df['content'] = df['content'].map(alphanumeric).map(punc_lower)
df['content'] = df['content'].fillna(value='')

content = df['content'].to_list()

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = ENGLISH_STOP_WORDS,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(content)

tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(content)

lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf.fit(dtm_tf)

lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(n_components=20, random_state=0)

In [3]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

2020-11-10 00:27:57,709 : INFO : NumExpr defaulting to 8 threads.


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16    -0.225163  0.117056       1        1  8.898440
15    -0.307007 -0.084572       2        1  7.230497
18    -0.050675  0.027401       3        1  5.336158
4      0.040184 -0.185134       4        1  5.283363
14     0.126129  0.100472       5        1  5.182833
17     0.122874  0.083855       6        1  5.157642
6     -0.199357 -0.121248       7        1  5.128173
8      0.171813 -0.154162       8        1  4.909331
12     0.047313 -0.098102       9        1  4.872114
19     0.059406  0.060289      10        1  4.750415
13    -0.029245 -0.110690      11        1  4.667550
2      0.115984 -0.014575      12        1  4.635199
9     -0.056154  0.087391      13        1  4.506762
7      0.105646  0.031143      14        1  4.466416
10     0.064240 -0.126178      15        1  4.464997
0      0.023605 -0.138205      16        1  4.329177
5     -0.026800  0.184073      17        1  4.252896
3      0.075937  0.193600      18        1  4.238342
1      0.035291  0.039950      19        1  4.035837
11    -0.094022  0.107635      20        1  3.653859, topic_info=               Term         Freq        Total Category  logprob  loglift
445         genshin  1824.000000  1824.000000  Default  30.0000  30.0000
556          impact  1253.000000  1253.000000  Default  29.0000  29.0000
173          childe   528.000000   528.000000  Default  28.0000  28.0000
202            come   526.000000   526.000000  Default  27.0000  27.0000
446   genshinimpact   497.000000   497.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
247       currently    18.344307    39.636039  Topic20  -4.7469   2.5390
1205         wishes    19.398979    52.898835  Topic20  -4.6910   2.3062
446   genshinimpact    27.487142   497.358232  Topic20  -4.3425   0.4138
634            live    26.936073   547.201940  Topic20  -4.3628   0.2980
475           grind    14.962316   101.316934  Topic20  -4.9507   1.3967

[967 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1         4  0.950100  absolutely
3         9  0.987989       abyss
4         5  0.861355         acc
4        15  0.061525         acc
5        10  0.029751     account
...     ...       ...         ...
1238     12  0.046185     zhongli
1238     13  0.002566     zhongli
1238     17  0.082108     zhongli
1238     18  0.084673     zhongli
1238     19  0.012829     zhongli

[2341 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 16, 19, 5, 15, 18, 7, 9, 13, 20, 14, 3, 10, 8, 11, 1, 6, 4, 2, 12])

In [4]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16    -0.201614  0.155057       1        1  9.371922
14     0.012722 -0.003247       2        1  5.387846
17     0.032335 -0.043418       3        1  5.303257
6     -0.097860 -0.108908       4        1  5.227688
13     0.083256 -0.080901       5        1  5.195659
18    -0.064637 -0.001131       6        1  5.132434
2      0.110337 -0.066378       7        1  5.121816
1      0.093945  0.037882       8        1  5.115390
12     0.057446 -0.116557       9        1  5.023184
9     -0.026734 -0.010124      10        1  4.988250
7      0.142044  0.063457      11        1  4.867995
15    -0.194514  0.042132      12        1  4.751215
4      0.036404 -0.023275      13        1  4.638827
8      0.084394 -0.055967      14        1  4.425401
5      0.035601  0.204132      15        1  4.335173
11    -0.189109  0.006787      16        1  4.319878
19     0.153462  0.194049      17        1  4.302493
0     -0.036226 -0.016573      18        1  4.262487
3      0.032806 -0.041800      19        1  4.136431
10    -0.064059 -0.135215      20        1  4.092652, topic_info=               Term        Freq       Total Category  logprob  loglift
634            live  177.000000  177.000000  Default  30.0000  30.0000
765          paimon   93.000000   93.000000  Default  29.0000  29.0000
446   genshinimpact  167.000000  167.000000  Default  28.0000  28.0000
202            come  145.000000  145.000000  Default  27.0000  27.0000
687        memepact   76.000000   76.000000  Default  26.0000  26.0000
...             ...         ...         ...      ...      ...      ...
590            klee   11.682113  165.513970  Topic20  -4.3860   0.5450
1141            use    8.939405   39.117778  Topic20  -4.6535   1.7199
518             hey    8.352411   31.143489  Topic20  -4.7215   1.8799
434            game    9.184390  154.650831  Topic20  -4.6265   0.3723
583            just    8.514065  195.240804  Topic20  -4.7023   0.0635

[973 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         2  0.610480        able
0         3  0.122096        able
0         4  0.122096        able
0        11  0.122096        able
1        13  0.929482  absolutely
...     ...       ...         ...
1238     14  0.040230     zhongli
1238     15  0.104597     zhongli
1238     17  0.104597     zhongli
1238     18  0.016092     zhongli
1239     17  0.858922      zombie

[2603 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 15, 18, 7, 14, 19, 3, 2, 13, 10, 8, 16, 5, 9, 6, 12, 20, 1, 4, 11])

In [5]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16    -0.408941 -0.179815       1        1  8.898440
15    -0.491661  0.083033       2        1  7.230497
18    -0.051972 -0.145862       3        1  5.336158
4     -0.144401 -0.363800       4        1  5.283363
14     0.067940  0.336083       5        1  5.182833
17    -0.070653  0.112307       6        1  5.157642
6     -0.343952  0.180488       7        1  5.128173
8     -0.244033 -0.288553       8        1  4.909331
12     0.021788 -0.417552       9        1  4.872114
19     0.293791 -0.347022      10        1  4.750415
13     0.131195 -0.292992      11        1  4.667550
2      0.237034  0.396496      12        1  4.635199
9     -0.278779 -0.010416      13        1  4.506762
7      0.128740  0.113173      14        1  4.466416
10    -0.211484  0.298477      15        1  4.464997
0     -0.097253  0.430383      16        1  4.329177
5      0.241403 -0.108269      17        1  4.252896
3      0.459129 -0.109208      18        1  4.238342
1      0.382290  0.065062      19        1  4.035837
11     0.379819  0.247988      20        1  3.653859, topic_info=               Term         Freq        Total Category  logprob  loglift
445         genshin  1824.000000  1824.000000  Default  30.0000  30.0000
556          impact  1253.000000  1253.000000  Default  29.0000  29.0000
173          childe   528.000000   528.000000  Default  28.0000  28.0000
202            come   526.000000   526.000000  Default  27.0000  27.0000
446   genshinimpact   497.000000   497.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
247       currently    18.344307    39.636039  Topic20  -4.7469   2.5390
1205         wishes    19.398979    52.898835  Topic20  -4.6910   2.3062
446   genshinimpact    27.487142   497.358232  Topic20  -4.3425   0.4138
634            live    26.936073   547.201940  Topic20  -4.3628   0.2980
475           grind    14.962316   101.316934  Topic20  -4.9507   1.3967

[967 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1         4  0.950100  absolutely
3         9  0.987989       abyss
4         5  0.861355         acc
4        15  0.061525         acc
5        10  0.029751     account
...     ...       ...         ...
1238     12  0.046185     zhongli
1238     13  0.002566     zhongli
1238     17  0.082108     zhongli
1238     18  0.084673     zhongli
1238     19  0.012829     zhongli

[2341 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 16, 19, 5, 15, 18, 7, 9, 13, 20, 14, 3, 10, 8, 11, 1, 6, 4, 2, 12])

In [6]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
16     -10.878068   10.538728       1        1  8.898440
15     -60.884026  -75.591820       2        1  7.230497
18      -2.336119  -53.289074       3        1  5.336158
4      -61.385319  -12.914634       4        1  5.283363
14     -28.737743  126.296997       5        1  5.182833
17       1.066606 -123.552811       6        1  5.157642
6       45.140720   -7.947869       7        1  5.128173
8     -140.795975  -98.530548       8        1  4.909331
12      61.531681  -83.240562       9        1  4.872114
19     -72.563057 -140.396881      10        1  4.750415
13      15.984491   63.578186      11        1  4.667550
2     -160.851624   68.945145      12        1  4.635199
9       85.961830   51.922935      13        1  4.506762
7      -99.020836  109.936348      14        1  4.466416
10    -176.764130   -8.681684      15        1  4.464997
0     -117.189880  -37.420200      16        1  4.329177
5      108.829918  -22.946203      17        1  4.252896
3       47.588097  121.934814      18        1  4.238342
1     -106.467834   30.176889      19        1  4.035837
11     -50.613804   59.328842      20        1  3.653859, topic_info=               Term         Freq        Total Category  logprob  loglift
445         genshin  1824.000000  1824.000000  Default  30.0000  30.0000
556          impact  1253.000000  1253.000000  Default  29.0000  29.0000
173          childe   528.000000   528.000000  Default  28.0000  28.0000
202            come   526.000000   526.000000  Default  27.0000  27.0000
446   genshinimpact   497.000000   497.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
247       currently    18.344307    39.636039  Topic20  -4.7469   2.5390
1205         wishes    19.398979    52.898835  Topic20  -4.6910   2.3062
446   genshinimpact    27.487142   497.358232  Topic20  -4.3425   0.4138
634            live    26.936073   547.201940  Topic20  -4.3628   0.2980
475           grind    14.962316   101.316934  Topic20  -4.9507   1.3967

[967 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1         4  0.950100  absolutely
3         9  0.987989       abyss
4         5  0.861355         acc
4        15  0.061525         acc
5        10  0.029751     account
...     ...       ...         ...
1238     12  0.046185     zhongli
1238     13  0.002566     zhongli
1238     17  0.082108     zhongli
1238     18  0.084673     zhongli
1238     19  0.012829     zhongli

[2341 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 16, 19, 5, 15, 18, 7, 9, 13, 20, 14, 3, 10, 8, 11, 1, 6, 4, 2, 12])